# **DATA UNDERSTANDING**

Sebuah Perusahaan yang bernama "northwind" bergerak dibidang bisnis pengiriman ekspor dan import khusus makanan dari seluruh penjuru dunia. Mereka ingin mengetahui gambaran umum tentang bisnis yang sedang mereka jalankan. Mereka memiliki database yang menunjukkan detail dari proses transakasi pengiriman produk makanan perusahan. Dari database tersebut, perusahaan ingin bisa mendapatkan wawasan atau pengetahuan, sehingga nantinya mereka dapat menerapkan strategi yang tepat untuk memperoleh keuntungan yang lebih dari saat ini. Pengetahuan yang ingin didapat berfokus pada Customers, Supplier, Products, Orders dan Employees.

Sumber Database: Northwind Database

Database yang dimiliki mempunyai 13 tabel, yaitu:
- Customers             : Menyimpan informasi tentang data pelanggan dari perusahaan.
- Customer Demographics : Belum ada data yang dibuat.
- Customer Demo         : Belum ada data yang dibuat.
- Products              : Menyimpan informasi tentang jenis produk yang dimiliki oleh perusahaan.
- Categories            : Menyimpan infomasi kategori dari setiap product. 
- Orders                : Menyimpan informasi jual-beli yang dilakukan oleh pelanggan.
- OrderDetails          : Menyimpan detail dari setiap pesanan yang dilakukan oleh pelanggan.
- Employees             : Menyimpan semua informasi karyawan.
- Employee Teritories   : Menyimpan informasi tentang tempat tinggal karyawan.
- Region                : Menyimpan informasi tentang regional wilayah tempat tinggal.
- Territories           : Menyimpan informasi tentang daftar wilayah tempat tinggal.
- Suppliers             : Menyimpan semua infomasi tentang data supplier.
- Shippers              : Menyimpan informasi tentang informasi jasa pengiriman yang digunakan oleh perusahaan.

Setiap tabel yang tertera pada database dapat terhubung, baik secara langsung maupun tidak langsung, sehingga setiap informasi dari database ini akan dapat saling berkaitan.

![](2021-12-28-07-00-35.png)

# **DATABASE**

## **Connecting To Database**

Bagian ini merupakan langkah awal untuk mulai melakukan proses analisis data. Pertama adalah membuat koneksi ke database di mana seperti yang sudah dijelaskan sebelumnya, database yang akan digunakan adalah database ``Northwind``. Dengan melakukan koneksi ini, kita dapat mengakses seluruh tabel yang ada pada database.


In [ ]:
# Import Modules

import pandas as pd
import numpy as np
import mysql.connector as sqlcon

In [ ]:
# Connect To Database

mydb = sqlcon.connect(
    host = 'localhost',
    user = 'root',
    passwd = 'semangat1945',
    database = 'northwind'
)
curs = mydb.cursor()

Selain melakukan koneksi ke database, hal lain yang perlu dilakukan pada bagian awal ini adalah membuat sebuah fungsi untuk menuliskan ``query`` yang kemudian akan disimpan dalam bentuk DataFrame. Tujuannya adalah agar data hasil ``query`` yang dibuat dapat disimpan dan digunakan untuk keperluan analisis data. Nantinya, di sini terdapat 2 query untuk mengambil data-data yang ada di database, bukan hanya berasal dari 1 tabel, tapi juga akan melibatkan hubungan antar tabel.

In [ ]:
# Query Function

def sql_table (query) :
    curs.execute (query)
    result = curs.fetchall()
    tabel1 = pd.DataFrame(result, columns=curs.column_names)
    return tabel1

## **Data Detail dari Order**

Data pertama ini merupakan data utama yang nantinya akan dianalisa lebih lanjut. Data ini merupakan gabungan dari 3 tabel, yaitu tabel ```Orders```, ```OrderDetails```, dan ```Products```. Masing-masing dari setiap tabel tersebut diambil beberapa kolomnya dan tidak diambil secara keseluruhan. Informasi-informasi yang dianggap penting saja lah yang diambil. Informasi yang diambil antara lain adalah :
- OrderID dari tabel orders
- OrderDate dari tabel orders
- ShippedDate dari tabel orders
- RequiredDate dari tabel orders
- ShipCity dari tabel orders
- ShipCountry dari tabel orders
- ProductID dari tabel orderdetails
- UnitPrice dari tabel orderdetails
- ProductName dari tabel Products

Semua informasi tersebut kemudian dijadikan dalam sebuah DataFrame yang nantinya akan diolah informasinya.

In [ ]:
# Query Data Frame

sampletable = sql_table('''
    select 	orders.OrderID, orders.OrderDate, orders.ShippedDate, orders.RequiredDate, orders.ShipCity, orders.ShipCountry, 
	orderdetails.ProductID, orderdetails.UnitPrice, products.ProductName
    from orders
    left join orderdetails on orders.OrderID = orderdetails.OrderID
    left join products on orderdetails.ProductID = products.ProductID;'''
)

# **DATA MANIPULATION**

In [ ]:
# memerika apakah ada tabel yang memiliki nilai null

print (sampletable.isnull().sum())

OrderID          0
OrderDate        0
ShippedDate     73
RequiredDate     0
ShipCity         0
ShipCountry      0
ProductID        0
UnitPrice        0
ProductName      0

In [ ]:
# menghapus baris/row yang memiliki nilai null

sampletable.dropna(inplace = True)  #<-Proses menghapus baris/row yang memiliki nilai Null

print (sampletable.isnull().sum())  #<-check kembali apakah tabel masih memiliki nilai Null

OrderID         0
OrderDate       0
ShippedDate     0
RequiredDate    0
ShipCity        0
ShipCountry     0
ProductID       0
UnitPrice       0
ProductName     0

In [ ]:
# print 5 baris teratas pada tabel

print (sampletable.head(5))

In [ ]:
# membuat tabel processing date yaitu waktu barang sampai ke pembeli setelah dikirim
sampletable['ProcessingDate'] = sampletable['RequiredDate'] - sampletable['ShippedDate'] 
#<-menghitung processing date dari pesanan

In [ ]:
# print (sampletable.sample(5))     
print (sampletable['ProcessingDate'].value_counts())
#<-setelah melihat kolom processing date, ternyata ada yang bernilai minus yang berarti pesanan sudah diterima customer bahkan
#<-sebelum pesanan dikirim yang mana itu tidak mungkin, oleh karena itu data tersebut dinilai tidak valid dan diasumsikan terdapat
#<-kesalahan input pada sistem. Data yang memiliki nilai minus pada kolom processingdate akan dihapus

In [ ]:
#menghapus row yang memiliki nilai minus pada kolom processingdate

#<-mencari dan menyimpan index baris pada tabel yang memiliki nilai minus pada kolom processingdate
indextobedeleted = sampletable[sampletable['ProcessingDate'].dt.days < 0].index 

#<-menghapus index yang memiliki nilai minus pada kolom processingdate
sampletable.drop(indextobedeleted, inplace = True)

#<-melihat kolom processing date setelah diproses ternyata sudah tidak memiliki nilai minus sehingga data dapat dikatakan sudah bersih
print (sampletable['ProcessingDate'].value_counts())                    